# AndanPerek

## Тема
Анализ каталога продуктов питания [Перекрестка](https://www.perekrestok.ru).

## Cбор данных

Все функции парсинга находятся в **parcer**

In [76]:
import parcer

Собираем все **категории** товаров, и ссылки на них

In [3]:
cats = parcer.parce_cats("https://www.perekrestok.ru/cat") # /ᐠ｡ꞈ｡ᐟ\
cats[0]

['Все на дачу', '/cat/mc/1181/vse-na-dacu']

Уберем категории, которые не относятся к продуктам питания

In [8]:
undesired_cats = [
    'Все на дачу',
    'Зоотовары',
    'Для дома и дачи',
    'Красота, гигиена, аптека',
    'Уборка',
    'Товары для мам и детей',
    'Системы нагревания, табак',
    'Посуда',
]

cats = [cat for cat in cats if cat[0] not in undesired_cats]

По ссылкам на категории, собираем **подкатегории** и ссылки на подкатегории

In [10]:
subcats = parcer.parce_subcats(cats)
subcats[0]

('Зелёная Линия', 'От Перекрёстка', '/cat/c/284/zelenaa-linia')

Уберем подкатегории, которые не относятся к продуктам питания

In [11]:
undesired_subcats = [
    'Home Story',
    'Kokoro',
    'Honey Kid',
]

subcats = [subcat for subcat in subcats if subcat[0] not in undesired_subcats]

Теперь благодаря ссылкам на подкатегории мы можем собрать все листинги. В листингах есть ссылки на товары, а также поверхностная информация

In [34]:
listing_products = parcer.parce_listing_products(subcats)
listing_products[0]

['Томаты черри на ветке, 250г',
 '/cat/150/p/tomaty-cerri-na-vetke-250g-3673307',
 'От Перекрёстка',
 'Маркет',
 '4.92',
 '250 гр',
 '179,99 ₽/шт',
 '179,99 ₽',
 '219,99 ₽',
 '-18%',
 '/cat/c/287/market']

In [72]:
parcer.save_listing_products_to_csv('data/listing_products.csv', listing_products)

Список успешно сохранен в файл data/listing_products.csv


In [75]:
products = parcer.parce_products(listing_products)
products[0]

['/cat/150/p/tomaty-cerri-na-vetke-250g-3673307',
 'Томаты черри на ветке, 250г',
 17999,
 '-18%',
 492,
 '15.0',
 '0.8 г',
 '0.1 г',
 '2.8 г',
 'Томаты.',
 'ТД Самарканд Агро',
 'Маркет',
 'Россия',
 '30 дней',
 '15 ℃',
 '5 ℃']

In [67]:
parcer.save_products_to_csv('data/products.csv', products)

Список успешно сохранен в файл data/products.csv


## Предварительныя обработка

In [78]:
import pandas as pd
import numpy as np

In [112]:
listing_products_df = pd.read_csv("data/listing_products.csv")
products_df = pd.read_csv("data/products.csv")

In [113]:
merged_df = pd.merge(products_df, listing_products_df, left_on='product_href', right_on='href' , how='left')

Так как в некоторых категориях товары дублировались, получилось так что мы собрали дубликаты, избавимся от них, а за одно и от лишних столбцов

In [114]:
merged_df.drop_duplicates(inplace=True)
merged_df.dropna(subset=['href'], inplace=True)
merged_df = merged_df.drop(['price_x', 'discount_x', 'title', 'product_href', 'subcat_href','rating_x'], axis=1) 

Переименовываем столбцы после слияния и сортируем

In [115]:
new_names = {
    'rating_y': 'rating',        
    'price_y': 'pricing',
    'discount_y': 'discount',
    'new_price': 'price',
    'new_price': 'price',
    'cat': 'category',
    'subcat': "subcategory",
}
merged_df = merged_df.rename(columns=new_names)

In [116]:
new_order = [
    'name',
    'category',
    'subcategory',
    'href',

    'kalories',
    'proteins',
    'fats',
    'carbohydrates',
    'composition',

    'manufacturer',
    'brand',
    'country',
    
    'rating',
    'size',
    'pricing',
    'price',
    'old_price',
    'discount',

    'shelf_life',
    'max_storage_temperature',
    'min_storage_temperature',
]
len(new_order)
merged_df = merged_df.reindex(columns=new_order)

In [117]:
merged_df

,name,category,subcategory,href,kalories,proteins,fats,carbohydrates,composition,manufacturer,...,country,rating,size,pricing,price,old_price,discount,shelf_life,max_storage_temperature,min_storage_temperature
0,Вино Мысхако Quintessence Zinfandel красное по...,Алкогольные напитки,Вино,/cat/2/p/vino-myshako-quintessence-zinfandel-k...,95.0,0.0 г,0.0 г,0.6 г,"Виноград сорта Зинфандель, пищевая добавка (ко...",Мысхако,...,Россия,4.89,750 мл,"899,99 ₽/шт","899,99 ₽","899,99 ₽",3 по цене 2-х,9999 дней,20 ℃,5 ℃
1,"Вино Takar Арени красное сухое 14%, 750мл",Алкогольные напитки,Вино,/cat/2/p/vino-takar-areni-krasnoe-suhoe-14-750...,74.0,0.0 г,0.0 г,0.3 г,Виноград сорта Арени; пищевая добавка (консерв...,Armenia Wine,...,Армения,4.90,750 мл,"699,99 ₽/шт","699,99 ₽","899,99 ₽",-22%,9999 дней,NaN,5 ℃
2,"Вино Terras de Pegoes красное сухое 12.5%, 750мл",Алкогольные напитки,Вино,/cat/2/p/vino-terras-de-pegoes-krasnoe-suhoe-1...,75.0,0.0 г,0.0 г,0.1 г,"Сортовой состав винограда: Каштелао - 60%, Али...",Cooperativa Agrícola de Santo Isidro De Pegões,...,Португалия,4.78,750 мл,"499,99 ₽/шт","499,99 ₽","699,99 ₽",-28%,9999 дней,NaN,NaN
3,"Вино Highway to Hell красное сухое 13%, 750мл",Алкогольные напитки,Вино,/cat/2/p/vino-highway-to-hell-krasnoe-suhoe-13...,79.0,0.0 г,0.0 г,0.0 г,"Виноград сорта Монастрель, пищевая добавка (ко...",Wines N'Roses,...,Испания,4.81,750 мл,"1 099,00 ₽/шт","1 099,00 ₽",NaN,Нет скидки,9999 дней,NaN,5 ℃
4,"Вино Don Garcia красное сухое 11%, 1л",Алкогольные напитки,Вино,/cat/2/p/vino-don-garcia-krasnoe-suhoe-11-1l-2...,0.0,0.0 г,0.0 г,0.0 г,"Красные сорта винограда, пищевая добавка (конс...",J. Garcia Carrion,...,Испания,4.75,1000 мл,"549,99 ₽/шт","549,99 ₽",NaN,Нет скидки,9999 дней,20 ℃,5 ℃
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25987,Мёд Берестов А.С. Башкирхан избранное липовый ...,"Мёд, варенье, джемы, сиропы",Мёд,/cat/112/p/med-berestov-a-s-baskirhan-izbranno...,321.0,0.0 г,0.0 г,80.3 г,Мед натуральный.,Берестов,...,Россия,4.87,500 гр,"799,99 ₽/шт","799,99 ₽",NaN,Нет скидки,730 дней,NaN,NaN
25988,"Сироп Имбирный с лимоном фреш, 250мл",К постному столу,Сладости и снеки,/cat/110/p/sirop-imbirnyj-s-limonom-fres-250ml...,196.0,0.0 г,0.0 г,49.0 г,"Водный экстракт корня имбиря с зеленым чаем, ф...",Биоинвентика,...,Россия,4.82,250 мл,"164,99 ₽/шт","164,99 ₽",NaN,Нет скидки,547 дней,NaN,NaN
25989,"Сироп Имбирный с лимоном фреш, 250мл","Мёд, варенье, джемы, сиропы",Сиропы,/cat/110/p/sirop-imbirnyj-s-limonom-fres-250ml...,196.0,0.0 г,0.0 г,49.0 г,"Водный экстракт корня имбиря с зеленым чаем, ф...",Биоинвентика,...,Россия,4.82,250 мл,"164,99 ₽/шт","164,99 ₽",NaN,Нет скидки,547 дней,NaN,NaN
26003,"Конфитюр Frueva из инжира, 310г",К постному столу,Сладости и снеки,/cat/775/p/konfitur-frueva-iz-inzira-310g-4068149,180.0,0.0 г,0.0 г,45.0 г,"Инжир, жидкий виноградный сахар, вода, лимонны...",Абрико,...,Россия,4.90,310 мл,"299,99 ₽/шт","299,99 ₽",NaN,Нет скидки,730 дней,25 ℃,0 ℃
